In [1]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pprint


service = EdgeService(executable_path=r"..\edgedriver_win64\msedgedriver.exe")
browser = webdriver.Edge(service=service)


In [2]:
browser.get("https://sellercenter.daraz.com.np/apps/seller/login")

In [4]:

phone_input = browser.find_element(By.XPATH, '//*[@id="account"]')
phone_input.send_keys('9811400555')

password_input = browser.find_element(By.XPATH, '//*[@id="password"]')
password_input.send_keys('Alam@5555')

login_button = browser.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[1]/div/div[2]/div/div/div[2]/form/div[3]/button')
login_button.click()


KeyboardInterrupt



In [7]:
def ul_elements(browser):
    try:
        ul_list=browser.find_element(By.CSS_SELECTOR, ".next-overlay-wrapper.opened").text.split('\n')
        return ul_list
    except:
        return []
            
            

In [8]:
def required_attribute(string):
    # Split the input string into lines
    lines = string.split('\n')

    # Extract elements that have '*' before them
    extracted_elements = [lines[i + 1].strip() for i in range(len(lines) - 1) if lines[i].strip() == '*']
    return extracted_elements

In [9]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException
import time


def attribute_extractor_obj(browser):
    wait = WebDriverWait(browser, 10)
    element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="root"]/section/div[2]/div/div[1]/div/div/div[2]/form/div/div[1]/div[2]/div/div/div/div[2]/div[1]/div[2]/button')))

    # Scroll the element into view and allow some time for the scrolling
    browser.execute_script("arguments[0].scrollIntoView(true);", element)
    time.sleep(2)

    # Try clicking with exception handling for intercepted clicks
    try:
        element.click()
        time.sleep(2)  # Extra wait after the click
    except ElementClickInterceptedException:
        print("Element click intercepted, retrying with JavaScript click.")
        # Retry the click using JavaScript if intercepted
        browser.execute_script("arguments[0].click();", element)
        time.sleep(2)
     # Allow time for the UI to update

    # Locate the grid containing attribute options
    grid = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/section/div[2]/div/div[1]/div/div/div[2]/form/div/div[1]/div[2]/div/div/div/div[2]/div[1]/div[1]/div")))

    all_fields=grid.text
    text = required_attribute(all_fields) or ['']
    attribute_obj = []
    if len(text)<=1:
         return [{
                "name": "Brand",
                "type": "list",
                "multiselect": False,
                "value": []
            }]
    else: 
        
        input = grid.find_elements(By.TAG_NAME, 'input')
        attribute_obj.append({
                "name": "Brand",
                "type": "list",
                "multiselect": False,
                "value": []
            })
        for i in range(1,len(text)):
                # Click the input field
            browser.execute_script("arguments[0].scrollIntoView(true);", input[i])  # Ensure element is in view
            input[i].click()
            time.sleep(3)  # Give time for dropdowns to appear

            # Determine the type of input field
            type_obj = 'list'
            value = ul_elements(browser)

            if 'Not Found' in value:
                value = []
            elif len(value) == 0:
                value = None
                type_obj = "text"

            # Check if the input field allows multiple selection
            multiselectable = 'aria-multiselectable="true"' in input[i].get_attribute('outerHTML')

            # Append attribute details to the list
            attribute_obj.append({
                "name": text[i],
                "type": type_obj,
                "multiselect": multiselectable,
                "value": value
            })
                

        return attribute_obj

In [10]:
import json
with open('search_error.json', 'w') as json_file:
    json.dump(attribute_extractor_obj(browser), json_file, indent=4)

In [8]:
import json

# Path to your JSON file
file_path = 'categories_reference.json'  # Adjust the path if necessary

# Load the JSON data
with open(file_path, 'r') as file:
    data = json.load(file)   

paths = []

def find_paths(d, current_path=None):
    if current_path is None:
        current_path = []
    
    for k, v in d.items():
        new_path = current_path + [k]  # Create a new path as a list
        if isinstance(v, dict) and v:  # Check if the value is a non-empty dict
            find_paths(v, new_path)
        else:  # Leaf node (empty dict)
            paths.append(new_path)  
find_paths(data)
path_str=[' > '.join(path) for path in paths]


In [21]:
import json
with open('Attributes_batch1.json', 'r') as json_file:
    data=json.load(json_file)
    print(path_str.index(data[-1]['Category']))

846


In [26]:
import json
with open('Attributes_batch2.json', 'r') as json_file:
    data2=json.load(json_file)
    print(path_str.index(data2[0]['Category']), path_str.index(data2[-1]['Category']))

847 999


In [27]:
import json

with open('Attributes_batch1.json', 'r') as file1, open('Attributes_batch2.json', 'r') as file2:
    batch1 = json.load(file1)
    batch2 = json.load(file2)

batch1.extend(batch2)

with open('Attribute_batch_0.json', 'w') as outfile:
    json.dump(batch1, outfile)

print("Merging completed and saved to Merged_Attributes.json")


Merging completed and saved to Merged_Attributes.json
